In [1]:
import pandas as pd
import numpy as np

import re, unicodedata
import pandas as pd

In [2]:


brizo_id_campaign_log = pd.read_csv('/Users/yukachen/marketing-operation/campaign_touch_points/brizo_id_campaign_log.csv')

In [3]:
display(brizo_id_campaign_log.head(5))

,brizo_id,list_source
0,b16872c1695c8f3d,2024-11—2nd DM - English_Other
1,fd839c0293eeafbf,2024-11—2nd DM - English_Other
2,06f64ace3ee27934,2024-11—2nd DM - English_Other
3,6711eba132ab3e4b,2024-11—2nd DM - English_Other
4,d6fae7175b6522d8,2024-11—2nd DM - English_Other


In [4]:
print(brizo_id_campaign_log['list_source'].unique())

brizo_id_campaign_log['list_source'].drop_duplicates().to_csv(
    'unique_list_source.csv', index=False, header=['list_source']
)


['2024-11—2nd DM - English_Other' '2024-04—Group C: CW Gift_SMS (Brizo)'
 '2025-07—C-LaborLowGMV' '2025-01—D - English'
 '2024-06—Group A: SQL_Trial (Brizo) 1-28314'
 '2025-03—B - Heartfelt (22407-44812)' '2025-05—Group A - High GMV'
 '2024-06—Group B: 50% (Brizo) 28315-57074'
 '2024-11—Unknown (28987-50734)'
 '2024-05—Group B: Bifold (Brizo) 28315-57074'
 '2024-02—Brizo - SQL Gift (G1)' '2024-09—Brizo A (Creative) 1-24229'
 '2024-11—Chinese (1-20118)' '2025-01—B - Chinese (Low GMV)'
 '2024-03—Group B - Trial Only' '2024-12—A - Chinese _Unknown'
 '2024-08—Brizo - A ($100) 1-18487' '2025-01—A - Chinese (High GMV)'
 '2025-04—Group A - Giftcard (1-25601)'
 '2024-08—Brizo - C ($300) 35527-51230' '2024-11—English (20119-28986)'
 '2024-07—Brizo A - Copy1  (1-25280)' '2024-10—A - Western 1-26130'
 '2025-07—B-RevenueHighGMV' '2025-01—C - Bilingual'
 '2025-06—Group B - Japanese' '2025-02—Drop1C - Bilingual VO'
 '2024-07—Brizo B - Copy2 (25281-51743)'
 '2024-01—Brizo - CAN (69821-78112)' '2025-0

In [5]:
def clean_list_source(s: str) -> str:
    if pd.isna(s):
        return s
    s = unicodedata.normalize('NFKC', str(s)).strip()

    # A. Normalize the date separator to "YYYY-MM—"
    s = re.sub(r'^(\d{4})[-/](\d{2})\s*[—-]\s*', r'\1-\2—', s)

    # B. Convert stray underscores to spaces
    s = s.replace('_', ' ')

    # C. Make "Group A:" -> "Group A -"
    s = re.sub(r'(Group [A-Z])\s*:\s*', r'\1 - ', s)

    # D. Ensure single spaces around hyphens between words (not touching ranges)
    s = re.sub(r'(?<=\w)\s*-\s*(?=\w)', ' - ', s)

    # E. Normalize “Promo(G3)” -> “Promo (G3)” and extra spaces before parens
    s = re.sub(r'\s*\(', ' (', s)      # at least one space before '('
    s = re.sub(r'\(\s+', '(', s)       # no space right after '('
    s = re.sub(r'\s+\)', ')', s)       # no space before ')'

    # F. If a numeric range trails the string (e.g., " 28315-57074"), move into parens
    s = re.sub(r'\s(\d+-\d+)$', r' (\1)', s)

    # G. Collapse multiple spaces
    s = re.sub(r'\s{2,}', ' ', s).strip()

    return s

# Apply
brizo_id_campaign_log['list_source_clean'] = brizo_id_campaign_log['list_source'].map(clean_list_source)


In [6]:
brizo_id_campaign_log.drop(columns=['list_source'], inplace=True)

In [7]:
brizo_id_campaign_log = brizo_id_campaign_log.rename(columns={'list_source_clean': 'campaign_name'})

In [8]:
brizo_id_campaign_log_clean = brizo_id_campaign_log.copy()

In [9]:
# create year and month columns from the cleaned campaign_name
brizo_id_campaign_log_clean['year'] = brizo_id_campaign_log_clean['campaign_name'].str.extract(r'(\d{4})')
brizo_id_campaign_log_clean['month'] = brizo_id_campaign_log_clean['campaign_name'].str.extract(r'\d{4}\s*-\s*(\d{2})')

# Optional: convert to integer
brizo_id_campaign_log_clean['year'] = brizo_id_campaign_log_clean['year'].astype(int)
brizo_id_campaign_log_clean['month'] = brizo_id_campaign_log_clean['month'].astype(int)

In [10]:
# Create 'cuisine' column
# brizo_id_campaign_log_clean['cuisine'] = np.select(
#     [
#         brizo_id_campaign_log_clean['campaign_name'].str.contains('2025-06—Group A - Chinese', case=False, na=False),
#         brizo_id_campaign_log_clean['campaign_name'].str.contains('2025-06—Group B - Japanese', case=False, na=False)
#     ],
#     ['Chinese', 'Japanese'],
#     default='No Segmentation'
# )

# Create 'gmv_level' column
brizo_id_campaign_log_clean['gmv_level'] = np.select(
    [
        brizo_id_campaign_log_clean['campaign_name'].str.contains('HighGMV', case=False, na=False) |
        brizo_id_campaign_log_clean['campaign_name'].str.contains('High GMV', case=False, na=False),
        brizo_id_campaign_log_clean['campaign_name'].str.contains('LowGMV', case=False, na=False) |
        brizo_id_campaign_log_clean['campaign_name'].str.contains('Low GMV', case=False, na=False)
    ],
    ['High GMV', 'Low GMV'],
    default='No Segmentation'
)

brizo_id_campaign_log_clean['dm_drop'] = np.select(
    [
        brizo_id_campaign_log_clean['campaign_name'].str.contains('Drop1', case=False, na=False),
        brizo_id_campaign_log_clean['campaign_name'].str.contains('Drop2', case=False, na=False) |
        brizo_id_campaign_log_clean['campaign_name'].str.contains('2nd', case=False, na=False)
    ],
    ['Drop 1', 'Drop 2'],
    default='Drop 1'
)

brizo_id_campaign_log_clean['campaign_name'] = brizo_id_campaign_log_clean['campaign_name'].apply(
    lambda x: re.sub(r'[\s–—-]+', '_', x)
)

In [11]:
brizo_id_campaign_log_clean.to_csv('/Users/yukachen/marketing-operation/campaign_touch_points/brizo_dm_send_log.csv', index=False)

In [12]:
# Sample 25 random rows
sample_df = brizo_id_campaign_log_clean.sample(n=25, random_state=42)

# Save to CSV
sample_df.to_csv('/Users/yukachen/marketing-operation/campaign_touch_points/brizo_id_campaign_log_sample.csv', index=False)

In [13]:
# brizo_id_campaign_log['present'] = True
# wide = (brizo_id_campaign_log.pivot_table(index='brizo_id',
#                        columns='list_source_clean',
#                        values='present',
#                        aggfunc='any',         # handles duplicates safely
#                        fill_value=False)
#           .reset_index()
#           .replace({True: 'TRUE', False: 'FALSE'}))

In [14]:
# wide.to_csv('brizo_frequency_database_design.csv', index=False, encoding='utf-8-sig')